## YTDLP-Drive-Telegram
#### By <a href="https://github.com/dishapatel010">@dishapatel010</a>
This Notebook uses the <b>YTDLP</b> library to download videos/playlists from YouTube in the highest resolution stream. <br> Also can move the video to the Google Drive & Upload to Telegram.

<a href="https://github.com/dishapatel010/YTDLP-Drive-Telegram">![Github Repo](https://img.shields.io/badge/View%20On%20Github-%23000000.svg?style=for-the-badge&logo=Github&logoColor=white)</a>

### Installing & Importing The Dependencies

In [ ]:
!pip install yt_dlp pyrogram tgcrypto
from google.colab import drive
import os, sys, shutil
import yt_dlp
import asyncio, nest_asyncio
from pyrogram import Client, errors

### Mounting To Google Drive

In [ ]:
drive.mount('/content/drive')

### Playlist Downloader

In [ ]:
url = "https://youtube.com/playlist?list=OLAK5uy_kbZybHHW0BwIQo5RLB4n2_gVcjS5wNVSo"
ffmpeg_path = "/usr/bin/ffmpeg"

ytdl_opts = {
    'outtmpl': 'downloads/%(title)s.%(ext)s',
    'format': 'best',
    'merge_output_format': 'mkv',
    'ffmpeg_location': ffmpeg_path,
    'postprocessors': [{
        'key': 'FFmpegVideoRemuxer',
        'preferedformat': 'mkv'
    }]
}

with yt_dlp.YoutubeDL(ytdl_opts) as ydl:
    info_dict = ydl.extract_info(url, download=True)
    video_title = info_dict.get('title', None)

print("Playlist downloaded successfully!")

### Telegram Upload

In [ ]:
# Set API credentials and chat ID
BOT_TOKEN = ""
CHAT_ID = ""
API_HASH = ""
API_ID = ""
# Set path to source folder
source_folder = os.path.join(os.getcwd(), "downloads")

async def main():
    # Get total number of files in folder
    total_files = len(os.listdir(source_folder))
    
    # Initialize progress message with status
    message_text = f"Uploading {total_files} files\n\n"
    progress_message = await app.send_message(chat_id=int(CHAT_ID), text=message_text)
    await app.pin_chat_message(chat_id=int(CHAT_ID), message_id=progress_message.id)

    # Loop through files in source folder
    for i, filename in enumerate(os.listdir(source_folder)):
        filepath = os.path.join(source_folder, filename)
        
        # Send file to Telegram chat using bot
        try:
            await app.send_video(chat_id=int(CHAT_ID), file_name=filename, caption=filename, video=filepath)
            
            # Update progress message with current file count
            message_text = f"Uploading {total_files} files:\n\n{i+1}/{total_files} files uploaded ({(i+1)*100/total_files:.2f}%)"
            await app.edit_message_text(chat_id=int(CHAT_ID), message_id=progress_message.id, text=message_text)
            
            # Wait for some time before sending the next video (optional)
            await asyncio.sleep(3)
            
        except errors.FloodWait as e:
            print(f"Waiting {e.x} seconds...")
            await asyncio.sleep(e.x)
            await app.send_video(chat_id=int(CHAT_ID), file_name=filename, caption=filename, video=filepath)
            
            # Update progress message with current file count
            message_text = f"Uploading {total_files} files:\n\n{i+1}/{total_files} files uploaded ({(i+1)*100/total_files:.2f}%)"
            await app.edit_message_text(chat_id=int(CHAT_ID), message_id=progress_message.id, text=message_text)

            # Wait for some time before sending the next video (optional)
            await asyncio.sleep(3)

        except Exception as e:
            print(f"Error: {e}")
            
    # Send completion message to Telegram chat
    completion_message = "All files uploaded successfully!"
    message_text = f"All files uploaded successfully!\n\n{total_files}/{total_files} files uploaded (100.00%)"
    await app.unpin_chat_message(chat_id=int(CHAT_ID), message_id=progress_message.id)
    await app.delete_messages(chat_id=int(CHAT_ID), message_ids=progress_message.id)
    await app.send_message(chat_id=int(CHAT_ID), text=message_text)

if __name__ == '__main__':
    # Initialize Pyrogram client and bot
    app = Client("ytdlpgdtg", api_id=API_ID, api_hash=API_HASH, bot_token=BOT_TOKEN)

    nest_asyncio.apply()
    async with app:
        await main()

### Move to Drive

In [ ]:
# Move downloaded video to /content/drive/MyDrive
source_folder = os.path.join(os.getcwd(), 'downloads')
destination_folder = '/content/drive/MyDrive/'

for file in os.listdir(source_folder):
    source_path = os.path.join(source_folder, file)
    destination_path = os.path.join(destination_folder, file)
    
    if not os.path.exists(destination_path):
        shutil.move(source_path, destination_path)
        print("Video moved to destination folder:", destination_folder)
    else:
        print("File already exists in destination folder:", destination_folder)

### Delete Download directory

In [ ]:
# Specify the path of the directory to be deleted
dir_path = "/content/downloads"

try:
  # Remove the directory using shutil.rmtree()
  shutil.rmtree(dir_path)
  print(f"{dir_path} has been deleted successfully.")
except OSError as e:
  print(f"Error: {dir_path} : {e.strerror}")